# Theano MNIST
Before start using this, please select `Cell` - `All Output` - `Clear` to clear the old results. See [Classifying MNIST digits using Logistic Regression](http://deeplearning.net/tutorial/logreg.html) for details of the tutorial.

# Loading MNIST training data

In [ ]:
import theano
import theano.tensor as T
import numpy

In [ ]:
# import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# Defining a Neural Network

In [ ]:
# initialize with 0 the weights W as a matrix of shape (n_in, n_out)
W = theano.shared(
    value=numpy.zeros(
        (784, 10),
        dtype=theano.config.floatX
    ),
    name='W',
    borrow=True
)

# initialize the biases b as a vector of n_out 0s
b = theano.shared(
    value=numpy.zeros(
        (10,),
        dtype=theano.config.floatX
    ),
    name='b',
    borrow=True
)

# generate symbolic variables for input (x and y represent a
# minibatch)
x = T.matrix('x')  # data, presented as rasterized images

y = T.nnet.softmax(T.dot(x, W) + b)
y

In [ ]:
#y_ = T.ivector('y')  # labels, presented as 1D vector of [int] labels
y_ = T.matrix('y')
cross_entropy = -T.sum(y_*T.log(y))

# Defining the Train Step

In [ ]:
# allocate symbolic variables for the data
index = T.lscalar()  # index to a [mini]batch

learning_rate = 0.1

# compute the gradient of cost with respect to theta = (W,b)
g_W = T.grad(cost=cross_entropy, wrt=W)
g_b = T.grad(cost=cross_entropy, wrt=b)

# start-snippet-3
# specify how to update the parameters of the model as a list of
# (variable, update expression) pairs.
updates = [(W, W - learning_rate * g_W),
           (b, b - learning_rate * g_b)]

train_model = theano.function(
    inputs=[x, y_],
    outputs=cross_entropy,
    updates=updates
)

train_model

# Do 1000 times of mini-batch training

In [ ]:
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)

    batch_xs = numpy.asarray(batch_xs, dtype=theano.config.floatX)
    batch_ys = numpy.asarray(batch_ys, dtype=theano.config.floatX)

    train_model(batch_xs, batch_ys)

# Test

In [ ]:
test_set_x = numpy.asarray(mnist.test.images, dtype=theano.config.floatX)
test_set_y = numpy.asarray(mnist.test.labels, dtype=theano.config.floatX)

correct_prediction = T.eq(T.argmax(y, 1), T.argmax(y_, 1))
accuracy = T.mean(T.cast(correct_prediction, 'float32'))
accuracy_f = theano.function(
  inputs=[x, y_],
  outputs=accuracy,
)
print(accuracy_f(test_set_x, test_set_y))